In [1]:
import pandas as pd

In [2]:
import re

In [3]:
import phonenumbers
from phonenumbers import (
    NumberParseException,
    is_possible_number,
    number_type,
    PhoneNumberType
)

In [4]:
def standardizePhone(phone_str: str, default_region: str):
    if pd.isnull(phone_str) or pd.isna(phone_str) or phone_str.strip().lower()=='nan' or phone_str.strip()=='' or phone_str=='0':
        return ''
    if pd.isnull(default_region) or pd.isna(default_region) or default_region.strip().lower()=='nan' or default_region.strip()=='' or default_region=='0':
        return ''
    else:
        try:
            pn = phonenumbers.parse(phone_str, default_region)
            if is_possible_number(pn):
                t = number_type(pn)
                if t == PhoneNumberType.MOBILE:
                    std = f"+{pn.country_code} {pn.national_number}"
                    # tag = "Mobile"
                elif t in (
                    PhoneNumberType.FIXED_LINE,
                    PhoneNumberType.FIXED_LINE_OR_MOBILE,
                    PhoneNumberType.TOLL_FREE
                ):
                    # std = format_number(pn, PhoneNumberFormat.INTERNATIONAL)
                    std = f"+{pn.country_code} {pn.national_number}"
                    # tag = "Landline"
                else:
                    # std = format_number(pn, PhoneNumberFormat.INTERNATIONAL)
                    std = f"+{pn.country_code} {pn.national_number}"
                    # tag = "Other"

                std = std.replace('-', ' ')
                std = re.sub(r'\s+', ' ', std).strip()
                return std

        except NumberParseException:
            pass

        digits = re.sub(r"\D", "", phone_str).lstrip("0")
        default_cc = phonenumbers.country_code_for_region(default_region)
        std = f"+{default_cc} {digits}"
        return std

In [5]:
def standardizeStreet(df, col):
    pattern = r"[^A-Za-z0-9\s\.,\-/']"
    df[col+' std'] = (
        df[col]
        .str.replace('&', 'and')
        .str.replace(pattern, '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', regex=True)
        .str.strip()
    )
    return df[col+' std']

In [21]:
# df = pd.read_excel('../5-june-fix/lfa1.xlsx', sheet_name='TELEPHONE 1')
df = pd.read_excel('../phone.xlsx', dtype=str)

In [22]:
df.columns

Index(['Country', 'phone'], dtype='object')

In [23]:
df

,Country,phone
0,US,6514145401


In [24]:
df['Phone std'] = df.apply(lambda row: standardizePhone(row['phone'], row['Country']), axis=1)

In [25]:
df

,Country,phone,Phone std
0,US,6514145401,+1 6514145401


In [18]:
df.to_csv('../phoneStd.csv')